In [1]:
import pandas as pd

In [2]:
def fix_smiles(smi_in):
    return smi_in.strip('[]').strip("'")

In [3]:
df_list = []
for split in ["random","scaffold"]:
    for task in ["st","mt"]:
        for i in range(0,5):
            dirname = f"data/Sol{i:03d}"
            if task == "st":
                df_test = pd.read_csv(f"{dirname}/{split}_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_ST/test_preds.csv")
            else:
                df_test = pd.read_csv(f"{dirname}/{split}_mt_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_MT/test_preds.csv")
            df_pred['SMILES'] = df_pred.smiles.apply(fix_smiles)
            df_pred.drop('smiles',axis=1,inplace=True)
            df_pred.rename(columns={"Sol": "Sol_pred"},inplace=True)
            df_test.Sol = df_test.Sol.astype(float)
            df_combo = pd.concat([df_test[["SMILES","Name","Sol"]],df_pred[["Sol_pred"]]],axis=1)
            df_combo['cv_cycle'] = i
            df_combo['split'] = split
            df_combo['method'] = f"chemprop_{task}"
            df_combo = df_combo[["cv_cycle","split","method","SMILES","Name","Sol","Sol_pred"]]
            df_list.append(df_combo)

In [4]:
df_out = pd.concat(df_list)
df_out.to_csv("Sol_regression_preds.csv",index=False)

In [5]:
df_out

,cv_cycle,split,method,SMILES,Name,Sol,Sol_pred
0,0,random,chemprop_st,COc1ccc2c(-c3ccccc3)cc3nnnn3c2c1,Mol3209,-0.265288,-0.117077
1,0,random,chemprop_st,Cc1ccc(CNC(=O)CN2CCC(n3nnc4cc(C)ccc43)CC2)cc1,Mol647,1.902668,1.882847
2,0,random,chemprop_st,CCc1nc(CN2C(=O)COc3ccc(F)cc32)cs1,Mol995,1.854473,2.098710
3,0,random,chemprop_st,CCOC(C)C(=O)N1CCC(n2c(=O)[nH]c3ccccc32)CC1,Mol1707,2.308977,2.231284
4,0,random,chemprop_st,Cc1cc(F)ccc1NC(=O)c1cccc2cccnc12,Mol3142,0.195826,1.191502
...,...,...,...,...,...,...,...
213,4,scaffold,chemprop_mt,Cc1c[nH]c(=O)n1-c1ccc(C(=O)Nc2ccc3ccccc3n2)cc1,Mol166,1.233822,1.295429
214,4,scaffold,chemprop_mt,O=c1n(CCc2ccncc2)nnn1-c1ccccc1,Mol309,1.936062,1.263523
215,4,scaffold,chemprop_mt,Cc1cc(C)nc(NC(=O)N(C)C2CCC2)c1,Mol958,2.247271,1.968439
216,4,scaffold,chemprop_mt,O=C(NCCc1csc(N2CCCC2)n1)N1CCSCC1,Mol1361,2.207115,2.066377


In [6]:
df_chemprop = pd.read_csv("Sol_regression_preds.csv")
df_lightgbm = pd.read_csv("lightgbm_regression_results.csv")
pd.concat([df_chemprop, df_lightgbm]).to_csv("Sol_regression_ChemProp_LightGBM.csv",index=False)